# Advanced Python: Building Scalable Applications

## Module 1

#### Threads, Processes, Generators and Coroutines
 - Introduction to generators, coroutines, threads and processes.
 - Concurrency Vs Parallelism: Choosing generator/coroutines Vs Threads/Processes

#### Python ```threading``` module: a deep-dive
 - Python threading module API
 - Creating and managing threads.
 - An overview on threading module.
 - Using the Thread class and the Timer class.
 - Active threads Vs. Daemon threads.
 - Helper functions in the threading module.


#### Concurrency using Generators

In [ ]:
r = range(10)
r

In [ ]:
for v in r:
    print(v)

In [ ]:
def testfn():
    print("Start of testfn function...")
    return 100
    print("Back inside testfn function...")

testfn()
testfn()

In [ ]:
def testfn():
    print("Start of testfn function...")
    yield 100
    print("Back inside testfn function...")

g = testfn()
g


In [ ]:
iter(g)

In [ ]:
a = {11, 22, 33, 44}
print(a, type(a))
for v in a:
    print(v)

In [ ]:
a = [11, 22, 33, 44]
print(a, type(a))
li = iter(a)
print(li)

next(li)

In [ ]:
next(li)

In [ ]:
a = [11, 22, 33, 44, 55]
for v in a:
    print(v)

# ----
iterator = iter(a)
try:
    while True:
        v = next(iterator)
        # Body of 'for-loop'
        print(v)
except StopIteration:
    pass


In [ ]:
def testfn():
    print("Start of testfn function...")
    yield 100
    print("Back inside testfn function...")
    yield "Hello"
    print("Back again inside testfn function...")
    yield
    print("Back one more time inside testfn function...")
    yield
    print("End of testfn")

g = testfn()
g


In [ ]:
for v in testfn():
    print("In for loop: v =", v)
    

In [ ]:
def testfn():
    print("Start of testfn...")
    yield 100
    print("Back inside testfn...")
    return 200

g = testfn()
g

In [ ]:
for v in testfn():
    print(v)

In [ ]:
def fib(n):
    a, b = 0, 1
    for _ in range(n):
        print(a, end=" ")
        a, b = b, a + b

fib(10) # 0 1 1 2 3 5 8 13 21 34

In [ ]:
def fib_list(n): # This is reusable - but not concurrent!
    series = [0, 1]
    for _ in range(n-2):
        series.append(series[-1] + series[-2])
    return series

fib_list(10) # [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

for i in fib_list(10):
    print(i, i*i)

In [ ]:
from time import sleep

def fib_list(n): # This is reusable - but not concurrent!
    series = [0, 1]
    for _ in range(n-2):
        sleep(1)
        series.append(series[-1] + series[-2])
    return series

#fib_list(10) # [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

print("Start...")
for i in fib_list(10):
    print(i, i*i)

In [ ]:
from time import sleep

def fib_gen(n): # This is reusable and concurrent!
    a, b = 0, 1
    for _ in range(n):
        sleep(1)
        yield a
        a, b = b, a + b

print("Start...")
for i in fib_gen(10):
    print(i, i*i)

#### Coroutines
Coroutines - are units of execution within a program that allows asynchronous execution with the aid of an event loop


#### Use-cases for Generators and Coroutines
  - Use generator for implement concurrent stream processing pipelines 
    (equivalent of Producer-Consumer patterns / pipelines)
  - Use coroutines for multiplexing I/O operations in a concurrent manner 
    (Asynchronous I/O operations)

Both Generators and Coroutines are suitable for cooperative multitasking workflows.

Coroutines are Python's implementation of "Green-Threads" / "User-Threads" / N:1 Threading Model.

### Introduction to Threads in Python (the ```threading``` module)

Threads in Python (threading module) provide a means of implementing
"Preemptive Multitasking" by leveraging the OS provided mechanisms for managing threads

This is also known as "Native Threading" / "OS-level Threading" / "Kernel-supported Threading" / Light-Weight-Processes (LWPs) / 1:1 Threading Model

The ```threading``` module provides a portable high-level API abstraction for creating and managing threads that works for Windows / Linux / MacOS / Other mainstream OS platforms.

The initial interface of threading module was heavily inspired from the Java's threading library.


In [ ]:
from threading import Thread

Scheduling and context-switching between threads are managed by the OS kernel (process/scheduler subsystem). The CPU usage by the OS to preempt and context-switch to another task (threads / process) is known as "scheduler latency".
